In [1]:
import math
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import functions from the scipy.signal module for signal processing
from scipy.signal import butter, filtfilt, find_peaks
import scipy.signal as signal
#import and mount the drive module to access files and directories from Google Drive


In [2]:
# Carga el clasificador Haarcascade preentrenado para detección de caras
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Carga el video desde tu computadora a Google Colab
video_filename = "Juan60.mp4"
# Abre el video
cap = cv2.VideoCapture(video_filename)
fps = cap.get(cv2.CAP_PROP_FPS)
fs = round(fps)
print(fs)

60


In [3]:
# Añadimos una función para reducir la resolución del frame
def resize_frame(frame, scale_percent):
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)

In [4]:

# Inicializa el seguimiento de la cara
tracker = cv2.TrackerKCF_create()
init_tracking = False

green_values = []
frame_count = 0  # Añadimos un contador de frames

# Asumimos que 'cap' y 'face_cascade' están definidos antes de este fragmento de código

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Reducimos el tamaño del frame a un 50%
    frame = resize_frame(frame, scale_percent=80)

    if not init_tracking:
        # Convertimos el frame a escala de grises solo si estamos en la fase de detección
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Modificamos los parámetros para posiblemente acelerar la detección
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.6, minNeighbors=5, minSize=(20, 20))

        if len(faces) > 0:
            x, y, w, h = faces[0]
            tracker.init(frame, (x, y, w, h))
            init_tracking = True
    else:
        # Incrementamos el contador de frames
        frame_count += 1

        success, bbox = tracker.update(frame)
        if success:  # Calculamos el valor medio del verde cada 10 frames
            x, y, w, h = map(int, bbox)
            face_roi = frame[y:y+h, x:x+w]
            mean_green_value = np.mean(face_roi[:, :, 1])
            green_values.append(mean_green_value)

    # Mostramos el fotograma actual
    #cv2.imshow('Frame', frame)

    # Procesamos menos frames por segundo en la visualización, para mejorar el rendimiento
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [5]:
nyquist = 0.5 * fs
low = 1.0 / nyquist
high = 1.6 / nyquist
b, a = signal.butter(1, [low, high], btype='band')
filtered_signal = signal.filtfilt(b, a, green_values)

# Identificación del pulso
frequencies = np.fft.rfftfreq(len(filtered_signal), 1/fs)
fft_values = np.fft.rfft(filtered_signal)
pulse_frequency = frequencies[np.argmax(np.abs(fft_values))]
pulse_rate = pulse_frequency * 60  # Convertir a latidos por minuto

print(f"Pulso cardíaco estimado: {pulse_rate:.2f} BPM")

Pulso cardíaco estimado: 83.82 BPM


In [5]:
cap.release()
cv2.destroyAllWindows()